## Caixabank Tech 2022
https://nuwe.io/challenge/data-science-cb

In [381]:
import pandas as pd
import numpy as np

#Data preprocessing

In [382]:
df=pd.read_csv('train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6554 entries, 0 to 6553
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       6554 non-null   object 
 1   Open       6421 non-null   float64
 2   High       6421 non-null   float64
 3   Low        6421 non-null   float64
 4   Close      6421 non-null   float64
 5   Adj Close  6421 non-null   float64
 6   Volume     6421 non-null   float64
 7   Target     6554 non-null   int64  
dtypes: float64(6), int64(1), object(1)
memory usage: 409.8+ KB


We set the rows have nan values, the target to nan too.



In [383]:
df['Target'][df['Open'].isna()]=df['Target'][df['Open'].isna()].apply(lambda x: np.nan)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


We define a function to parse the date to float

In [384]:
from datetime import datetime

def datetime_to_float(d):
  datetime_object = datetime.strptime(d, '%Y-%M-%d')
  datetime_object.replace(hour=0, minute=0, second=0)
  return datetime_object.timestamp()
df['ParseDate']=df['Date'].apply(lambda x: datetime_to_float(x))

In [385]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Target,ParseDate
0,1994-01-03,3615.199951,3654.699951,3581.000000,3654.500000,3654.496338,0.0,0.0,757555260.0
1,1994-01-04,3654.500000,3675.500000,3625.100098,3630.300049,3630.296387,0.0,1.0,757641660.0
2,1994-01-05,3625.199951,3625.199951,3583.399902,3621.199951,3621.196289,0.0,1.0,757728060.0
3,1994-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,757814460.0
4,1994-01-07,3621.199951,3644.399902,3598.699951,3636.399902,3636.396240,0.0,1.0,757900860.0


Here we impute the remaining values, using kneighbors with a custom function so it uses the parsed dated, as metric to choose neighboor.

In [386]:
from sklearn.impute import KNNImputer
columns=['ParseDate','Open', 'High', 'Low', 'Volume','Close','Adj Close']
def mydist2 (x,y,**kwargs):
  return abs(x[0]-y[0])
imputer = KNNImputer(n_neighbors=3, weights="uniform",metric=mydist2)
df[columns]=imputer.fit_transform(df[columns])
df[columns].head(20)


,ParseDate,Open,High,Low,Volume,Close,Adj Close
0,757555260.0,3615.199951,3654.699951,3581.000000,0.0,3654.500000,3654.496338
1,757641660.0,3654.500000,3675.500000,3625.100098,0.0,3630.300049,3630.296387
2,757728060.0,3625.199951,3625.199951,3583.399902,0.0,3621.199951,3621.196289
3,757814460.0,3483.066650,3524.266601,3472.833333,0.0,3492.533366,3492.529785
4,757900860.0,3621.199951,3644.399902,3598.699951,0.0,3636.399902,3636.396240
5,758160060.0,3655.199951,3678.199951,3655.199951,0.0,3660.600098,3660.596436
6,758246460.0,3679.699951,3712.500000,3679.699951,0.0,3712.399902,3712.396240
7,758332860.0,3712.300049,3712.300049,3675.899902,0.0,3680.100098,3680.096436
8,758419260.0,3680.100098,3698.199951,3670.399902,0.0,3680.800049,3680.796387
9,758505660.0,3680.800049,3737.399902,3662.899902,0.0,3736.399902,3736.395996


Here we check the imputed values, checking the target field, if it's nan, that means that was imputed.

If the row was imputed, we check the Row - 3 to check if the Target has a valid value. If it isnt, we just set the row fields ('Close', 'Adj Close') as the Row -3 value, in case of it is under, in case that it has to be above the 'Close' value we will plus 1 to the value.

In [387]:
for i in range(0, len(df)):
    if(np.isnan(df.iloc[i].Target)):
      if(df.iloc[i-3]['Target'] and df.iloc[i-3]['Close']>=df.iloc[i]['Close']):
        df.loc[i,'Close']=df.iloc[i-3]['Close']+1
        df.loc[i,'Adj Close']=df.iloc[i-3]['Adj Close']+1
      elif(df.iloc[i-3]['Target']==0 and df.iloc[i-3]['Close']<df.iloc[i]['Close']):
        df.loc[i,'Close']=df.iloc[i-3]['Close']
        df.loc[i,'Adj Close']=df.iloc[i-3]['Adj Close']
      df.loc[i,'Target']=1 if df.iloc[i+3]['Close']>df.iloc[i]['Close'] else 0


Here we check that we filled all the missing values.

In [388]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6554 entries, 0 to 6553
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       6554 non-null   object 
 1   Open       6554 non-null   float64
 2   High       6554 non-null   float64
 3   Low        6554 non-null   float64
 4   Close      6554 non-null   float64
 5   Adj Close  6554 non-null   float64
 6   Volume     6554 non-null   float64
 7   Target     6554 non-null   float64
 8   ParseDate  6554 non-null   float64
dtypes: float64(8), object(1)
memory usage: 461.0+ KB


#Creating the model
We will be using a RNN, because it has an artificial neural network called LSTM. 
And it works well on temporary series.

In [397]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.layers import Dropout

First we are going to standarize the variables with StandardScaler

In [390]:
scaler = StandardScaler()
dataset = scaler.fit_transform(df[columns].drop('ParseDate',axis=1))
y=df['Target']

Here we are going to create the Train array, and it will contain in each day row of the matrix, the 30 previous days to make a prediction.

In [391]:
X_train = []
y_train = []

n_past = 30     # Number of past days we want to use to predict the future

for i in range(n_past, len(dataset)):
    X_train.append(dataset[i - n_past:i, 0:df.shape[1] - 1])
    y_train.append(y[i- 1:i])

X_train, y_train = np.array(X_train), np.array(y_train)

print('X_train shape == {}.'.format(X_train.shape))
print('y_train shape == {}.'.format(y_train.shape))

X_train shape == (6524, 30, 6).
y_train shape == (6524, 1).


As i mentioned above, we add to our model two layers of LSTM, after that it will be going a layer of Dropout to prevent the overfit. The last node uses the sigmoid activation function that will squeeze all the values between 0 and 1.

In [393]:
model = Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(n_past, df.shape[1]-3)))
model.add(LSTM(units=16, return_sequences=False))
model.add(Dropout(0.15))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, shuffle=True, epochs=30, validation_split=0.2, verbose=1, batch_size=256)


Epoch 1/30
21/21 [==============================] - 6s 133ms/step - loss: 0.6931 - accuracy: 0.5300 - val_loss: 0.6912 - val_accuracy: 0.5326
Epoch 2/30
21/21 [==============================] - 2s 92ms/step - loss: 0.6915 - accuracy: 0.5290 - val_loss: 0.6914 - val_accuracy: 0.5157
Epoch 3/30
21/21 [==============================] - 2s 91ms/step - loss: 0.6907 - accuracy: 0.5348 - val_loss: 0.6915 - val_accuracy: 0.5211
Epoch 4/30
21/21 [==============================] - 2s 92ms/step - loss: 0.6910 - accuracy: 0.5384 - val_loss: 0.6918 - val_accuracy: 0.5157
Epoch 5/30
21/21 [==============================] - 2s 90ms/step - loss: 0.6901 - accuracy: 0.5417 - val_loss: 0.6908 - val_accuracy: 0.5165
Epoch 6/30
21/21 [==============================] - 2s 89ms/step - loss: 0.6902 - accuracy: 0.5484 - val_loss: 0.6915 - val_accuracy: 0.5188
Epoch 7/30
21/21 [==============================] - 2s 89ms/step - loss: 0.6904 - accuracy: 0.5413 - val_loss: 0.6906 - val_accuracy: 0.5218
Epoch 8/30
2

We do the same with the test file, but i added the last days to our test matrix

In [498]:
datasetTest=pd.read_csv('test_x.csv')
datasetTestScaled = scaler.fit_transform(datasetTest.drop(['Date','test_index'],axis=1))

X_test = []
for i in range(n_past, len(datasetTestScaled)):
    X_test.append(datasetTestScaled[i - n_past:i])
for i in range(0,n_past):
  X_test[i]=X_train[(X_train.shape[0]-(n_past-i))]
X_test=np.array(X_test)

#Making the prediction

In [499]:
prediction=model.predict(X_test)


In [506]:
df = pd.DataFrame(prediction, 
             columns=['Target'])
df['test_index']=datasetTest['test_index']

df1 = df.pop('Target') 
df['Target']= df1.apply(lambda x: x>0.5)


,test_index,Target
0,6557,True
1,6558,True
2,6559,True
3,6560,True
4,6561,True
...,...,...
691,7248,True
692,7249,True
693,7250,True
694,7251,True


In [510]:
from pathlib import Path  
filepath = Path('predictions.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath) 

In [511]:
df.to_json(r'predictions.json')
